In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver



def multiply_values(a, b):
    """
    Multiply two values and return the result.

    Parameters:
        a (float): The first value.
        b (float): The second value.

    Returns:
        float: The product of a and b.
    """
    return a * b

llm = ChatOpenAI(model="gpt-4o-mini")
toolbox = [multiply_values]
llm = llm.bind_tools(toolbox)


# System message
chatbot_system_message = SystemMessage(content=("""
You are a helpful and knowledgeable chatbot assistant. 
Your goal is to provide clear and accurate answers to user questions based on the information they provide. 
Stay focused, concise, and ensure your responses are relevant to the context of the conversation. 
If you don’t have enough information, ask for clarification.”
"""))

# Nodes
def chatbot(state: MessagesState):
   return {"messages": [llm.invoke([chatbot_system_message] + state["messages"])]}

def summarization(state: MessagesState):
    summary_message = HumanMessage(content="""
    Summarize the above conversation while preserving full context, key points, and user intent.
    Only return the summarized content. Do not add explanations, section headers, or extra commentary.
    """)
    
    return {"messages": [llm.invoke(state["messages"] + [summary_message])]}


#Edges
def should_summarize(state: MessagesState) -> bool:
    return len(state["messages"]) > 2

def extended_condition(state: MessagesState):
    # First, check if the chatbot indicates a tool call
    if tools_condition(state) == "tools":
        return "tools"
    
    # Then, if no tool is needed, decide if summarization is needed
    if should_summarize(state):
        return "summarization"
    
    # Otherwise, finish the graph
    return END

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("chatbot", chatbot)
builder.add_node("tools", ToolNode(toolbox))
builder.add_node("summarization", summarization)

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", extended_condition)
builder.add_edge("tools", "chatbot")
builder.add_edge("summarization", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

# Show
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))


NameError: name 'END' is not defined

In [13]:
config = {"configurable": {"thread_id": 1}}
messages = [HumanMessage(content="What is 2 multiplied by 3?")]
messages = graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

What is 2 multiplied by 3?
================================== Ai Message ==================================
Tool Calls:
  multiply_values (call_gEEvxK6q9p6rTGxdu0b5JAje)
 Call ID: call_gEEvxK6q9p6rTGxdu0b5JAje
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply_values

6
================================== Ai Message ==================================

2 multiplied by 3 is 6.
